---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [3]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [5]:
def answer_one():
    return df['Gold'].argmax()

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [11]:
def answer_two():
    df['diff'] = abs(df['Gold'] - df['Gold.1'])
    return df['diff'].argmax()

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [26]:
def answer_three():
    won_summer = df['Gold'] >= 1
    won_winter = df['Gold.1'] >= 1
    new_df = df[won_summer & won_winter]
    new_df['diff'] = (abs(new_df['Gold'] - new_df['Gold.1']))/new_df['Gold.2']
    return new_df['diff'].argmax()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [33]:
def answer_four():
    df_copy = df.copy()
    df_copy['Gold.2'] = df_copy['Gold.2']*3
    df_copy['Silver.2'] = df_copy['Silver.2']*2
    df_copy['TotalPoints'] = df_copy['Gold.2'] + df_copy['Silver.2'] + df_copy['Bronze.2']
    Points = pd.Series(index=df.index.values.tolist(), data=df_copy['TotalPoints'])
    return Points

Afghanistan                             2
Algeria                               147
Argentina                             562
Armenia                                40
Australasia                            94
Australia                            4379
Austria                              2417
Azerbaijan                            187
Bahamas                               144
Bahrain                                 1
Barbados                                1
Belarus                               586
Belgium                              1188
Bermuda                                 1
Bohemia                                 5
Botswana                                2
Brazil                                736
British West Indies                     2
Bulgaria                             1659
Burundi                                27
Cameroon                               84
Canada                               3750
Chile                                  72
China                             

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [4]:
import pandas as pd
census_df = pd.read_csv('census.csv')
census_df.head()

IOError: File census.csv does not exist

In [2]:
def answer_five():
    # filter SUMLEV=50 for county only
    county_list = census_df['SUMLEV'] == 50
    new_cdf = census_df[county_list]
    return new_cdf['STNAME'].value_counts().idxmax() 

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [10]:
def answer_six():
    # sort based on CENSUS2010POP
    county_list = census_df['SUMLEV'] == 50
    new_cdf = census_df[county_list].sort_values(['CENSUS2010POP'], ascending=False)
    #return new_cdf['CTYNAME'][:3].tolist()
    return new_cdf['CTYNAME'][]
answer_six()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
209,50,4,9,6,37,California,Los Angeles County,9818605,9818700,9826009,...,-4.999439,-4.071266,-4.332318,-5.394299,-6.027694,-0.007808,0.377510,0.578049,0.215038,-0.437481
624,50,2,3,17,31,Illinois,Cook County,5194675,5195022,5199303,...,-6.399085,-5.939523,-6.011973,-9.374552,-10.561919,-3.000501,-2.669330,-2.358718,-5.102980,-6.263994
2667,50,3,7,48,201,Texas,Harris County,4092459,4093076,4108187,...,1.403957,3.518819,4.820913,5.243613,3.794291,7.154417,8.915715,10.683269,11.850629,10.303814
106,50,4,8,4,13,Arizona,Maricopa County,3817117,3817357,3825597,...,2.444413,9.232973,7.879873,8.318210,9.123309,4.726436,11.527593,10.390177,11.203958,11.985998
227,50,4,9,6,73,California,San Diego County,3095313,3095308,3104386,...,0.354751,0.235513,-0.246567,0.657077,-2.855045,4.382515,5.773072,4.745940,5.921091,2.770965
220,50,4,9,6,59,California,Orange County,3010232,3010266,3017866,...,1.992772,0.618346,-0.491825,-2.374727,-3.182714,6.033306,4.395138,3.681764,2.407607,1.587398
1884,50,1,2,36,47,New York,Kings County,2504700,2504710,2510335,...,-6.116225,-7.066189,-8.271203,-12.671578,-12.425023,3.070971,1.380027,0.899094,-2.260948,-2.107864
372,50,3,5,12,86,Florida,Miami-Dade County,2496435,2498017,2508171,...,7.464259,-6.908149,-8.130040,-10.464276,-12.205475,23.110902,6.891198,6.633403,6.087936,4.154777
2623,50,3,7,48,113,Texas,Dallas County,2368139,2367643,2373464,...,0.646423,4.853046,-2.117265,-0.754784,-0.981666,5.233271,9.100284,2.574144,4.639783,4.397784
1901,50,1,2,36,81,New York,Queens County,2230722,2230541,2235573,...,-7.701619,-9.309401,-7.554518,-11.787697,-10.891968,3.748957,1.318590,3.998295,1.244451,2.001030


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [68]:
def answer_seven():
    # filter only counties
    county_list = census_df['SUMLEV'] == 50
    new_cdf = census_df[county_list]
    # find max and min
    max_pop = new_cdf[["POPESTIMATE2010", "POPESTIMATE2011", "POPESTIMATE2012", "POPESTIMATE2013", "POPESTIMATE2014", "POPESTIMATE2015"]].max(axis=1)
    min_pop = new_cdf[["POPESTIMATE2010", "POPESTIMATE2011", "POPESTIMATE2012", "POPESTIMATE2013", "POPESTIMATE2014", "POPESTIMATE2015"]].min(axis=1)
    new_cdf['ABS_CHANGE'] = max_pop-min_pop
    # sort from biggest to smallest change
    new_cdf = new_cdf.sort_values(['ABS_CHANGE'], ascending=False)
    return new_cdf['CTYNAME'].iloc[0]

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [76]:
def answer_eight():
    # filter counties in regions 1 or 2
    new_cdf = census_df[(census_df['REGION']==1) | (census_df['REGION']==2)]
    # CTYNAME starts with 'Washington'
    new_cdf = new_cdf[new_cdf['CTYNAME'].str.contains('Washington')]
    # POPESTIMATE2015 > POPESTIMATE2014
    new_cdf = new_cdf[new_cdf['POPESTIMATE2015'] > new_cdf['POPESTIMATE2014']]
    final = new_cdf[['STNAME', 'CTYNAME']]
    return final

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
